Integrantes: 

* Daniel Julián Rojas Cruz
* Johan Tanta Villanueva

# Redes Bayesianas - Inteligencia Artificial


Se presenta la implementación de la función count que retorna el número de filas que satisfacen las igualdades


In [13]:
import pandas as pd
import numpy as np


df = pd.DataFrame(np.random.randint(0,10,size=(10, 3)), columns=list('ABC'))


def count(dataframe, index_list, values_list):
  resultado = dataframe
  for i in range(len(index_list)):
    index = index_list[i]
    value = values_list[i]

    #filtrar la columna 
    resultado = resultado[resultado[index] == value]
  c =resultado.shape[0]
  return c



## Objetivo 1
El objetivo 1 consiste en estimar la marginal para una variable en consulta a partir de un dataset

### Implementación del Objetivo 1


In [14]:

# Esta función recive el nombre del archivo CSV y devuelve un dataframe de Pandas
def leer_dataset(nom_archivo):
    return pd.read_csv(nom_archivo)


def cardinalidad(df, X):
    return df[X].unique()


# Esta funcióm devuelve un reporte del factor que contiene a la variable X
def estimar_marginal(df,X, alpha):
    card = cardinalidad(df,X) # lista de valores únicos
    
    nfilas = df.shape[0]
    
    
    probList = []
    for valor in card:
        m = count(df,[X],[valor])
        p = (m + alpha)/(nfilas + len(card)*alpha)
        probList.append(p)
    probColName = "P(" + X + ")"
    columns = {X:card, probColName: probList}
    
    
    return pd.DataFrame(columns)



df = leer_dataset("weather - weather.csv")
marginal = estimar_marginal(df,'play',1)
marginal

,play,P(play)
0,yes,0.625
1,no,0.375


## Objetivo 2
El objetivo 2 consiste en estimar la condicional para un par o más variables en consulta a partir del dataset previo

### Implementación del Objetivo 2


In [34]:


def estimar_condicional(df, X, Y, alpha):
    # hallar el número total de filas 
    nFilasResult = 1
    nfilas = df.shape[0]
    cardTarget = cardinalidad(df,X)
    nFilasResult = nFilasResult*len(cardTarget)
    
    
    cardY = {}
    for var in Y:
        cardY[var] = cardinalidad(df,var)
        nFilasResult = nFilasResult*len(cardY[var])
    
    
    
    columns = {}
    
    n_rep = 1
    
    colTarget = [cardTarget[i%len(cardTarget)] for i in range(nFilasResult)]
    
    columns[X] = colTarget
    n_rep = n_rep * len(cardTarget)
    
    for var, card in cardY.items():
        columns[var] = [ card[int(i/n_rep)%len(card)] for i in range(nFilasResult) ]
        
        n_rep = n_rep * len(card)

    
    result = pd.DataFrame(columns)
    probList = []
    
    prodCardY = 1
    for var in Y:
        prodCardY = prodCardY* len(cardY[var])
        
    
    #Iteramos por cada fila del dataframe y hallamos la probabilidad condicional
    for i in result.index:
        valTarget = result[X][i]
    
        
        valPadres = []
        for var in Y:
            valPadres.append(result[var][i])
      
        totalVar = [i for i in Y]
        totalVar.append(X)
        totalVal = valPadres
        totalVal.append(valTarget)
        
        
        prob_X_Y = (count(df,totalVar,totalVal) + alpha) / (nfilas + prodCardY*len(cardTarget)*alpha) 
        
        prob_Y = (count(df,Y,valPadres) + alpha) / (nfilas + prodCardY*alpha)
        p = prob_X_Y/prob_Y
        probList.append(p)
    # normalizamos las probabilidades
    
    for i in range(int (nFilasResult / len(cardTarget))):
        suma = 0
        for j in range(len(cardTarget)):
            suma = suma + probList[i*len(cardTarget) + j]
        
        for j in range(len(cardTarget)):
            probList[i*len(cardTarget) + j] = probList[i*len(cardTarget) + j] / suma 
    
    result['P'] = probList
    return result

cond = estimar_condicional(df,'outlook', ['windy','play'],1)
cond

,outlook,windy,play,P
0,overcast,False,yes,0.333333
1,rainy,False,yes,0.444444
2,sunny,False,yes,0.222222
3,overcast,True,yes,0.500000
4,rainy,True,yes,0.166667
5,sunny,True,yes,0.333333
6,overcast,False,no,0.200000
7,rainy,False,no,0.200000
8,sunny,False,no,0.600000
9,overcast,True,no,0.166667
